# L7: Build a Crew to Tailor Job Applications

In this lesson, you will built your first multi-agent system.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./fake_resume.md')
semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md')

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  3.67it/s]


- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

In [5]:
# from IPython.display import Markdown, display
# display(Markdown("./fake_resume.md"))

## Creating Agents

In [6]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [7]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [8]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [9]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## Creating Tasks

In [10]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [11]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [12]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [13]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Creating the Crew

In [14]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [15]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [16]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/joaomdmoura) URLs, and personal write-up (Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his abili

Action: Read website content
Action Input: {"website_url": "https://github.com/joaomdmoura"} 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



Thought: I need to gather information from the GitHub profile and personal write-up provided to create a comprehensive personal and professional profile for Noah.

Action: Read website content
Action Input: {"website_url": "https://github.com/joaomdmoura"} 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



Action: Search the internet
Action Input: {"search_query": "João Moura GitHub"}Final Answer: 
- Proven experience as a Full Stack Developer or similar role
- Experience developing desktop and mobile applications
- Familiarity with common stacks
- Knowledge of multiple front-end languages and libraries (e.g. HTML/ CSS, JavaScript, XML, jQuery)
- Knowledge of multiple object-oriented languages (e.g. Java, Python) and multiple framewor

Final Answer:
# Noah Williams
- Email: noah.williams@example.dev
- Phone: +44 11 111 11111

## Summary
Noah Williams is a distinguished Software Engineering Leader with an 18-year tenure in the technology industry, specializing in managing remote and in-office engineering teams. He possesses expertise in software development, process innovation, and team collaboration. Proficient in programming languages such as Ruby, Python, JavaScript, TypeScript, and Elixir, Noah has a strong background in data science and machine learning, enabling him to drive successful tech initiatives and startups. His educational background includes a joint PhD and MSc in Robotics and a diploma in Mechanical Engineering.

## Work Experience

### Director of Software Engineering, DataKernel (2022 - Present)
- Transformed the engineering division into a key revenue pillar, expanding the customer base and integrating cutting-edge AI technologies.
- Spearheaded strategic initiatives and set new industry benchmarks

Final Answer:

Based on the tailored resume of Noah Williams and the job requirements, here are the potential interview questions and talking points for the candidate to prepare for the initial interview:

1. Can you elaborate on your experience in transforming engineering divisions into revenue pillars, as mentioned in your work at DataKernel?
2. How have you integrated cutting-edge AI technologies and scalable databases to enhance product capabilities and market positioning?
3. In what ways have you fostered skill development and achieved strategic project goals within your teams?
4. Could you share insights into your role in defining long-term strategic initiatives and setting industry benchmarks in adopting AI technologies?
5. Describe your approach to leading diverse engineering teams across multiple time zones and the strategies you use to ensure transparency and mutual respect.
6. How do you recruit, mentor, and retain top talent to create a culture of continuous improvement and

- Dislplay the generated `tailored_resume.md` file.

In [17]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

# Noah Williams
- Email: noah.williams@example.dev
- Phone: +44 11 111 11111

## Summary
Noah Williams is a distinguished Software Engineering Leader with an 18-year tenure in the technology industry, specializing in managing remote and in-office engineering teams. He possesses expertise in software development, process innovation, and team collaboration. Proficient in programming languages such as Ruby, Python, JavaScript, TypeScript, and Elixir, Noah has a strong background in data science and machine learning, enabling him to drive successful tech initiatives and startups. His educational background includes a joint PhD and MSc in Robotics and a diploma in Mechanical Engineering.

## Work Experience

### Director of Software Engineering, DataKernel (2022 - Present)
- Transformed the engineering division into a key revenue pillar, expanding the customer base and integrating cutting-edge AI technologies.
- Spearheaded strategic initiatives and set new industry benchmarks in adopting AI technologies.

### Senior Software Engineering Manager, DataKernel (2019 - 2022)
- Directed engineering strategy in collaboration with C-level executives, managing diverse teams across multiple time zones.
- Recruited, mentored, and retained top talent, fostering a culture of continuous improvement.

### Founder & CEO, InnovPet (2019 - 2022)
- Founded InnovPet, a startup focused on innovative IoT solutions for pet care, overseeing product development and successful market entry.
- Set up an advisory board, established production facilities overseas, and navigated the company through a successful initial funding phase.

### Engineering Manager, EliteDevs (2018 - 2019)
- Formulated and executed strategic plans, managed multiple engineering teams, and implemented goal-setting frameworks aligned with company goals.

### Engineering Manager, PrintPack (2016 - 2018)
- Led a high-performance engineering team that increased company revenue by 500% within two years.
- Integrated data analytics into business decision-making processes, revolutionizing customer behavior analysis.

### Senior Software Engineer, DriveAI (2015 - 2016)
- Developed and optimized a central API that improved functionality for a large engineering team and thousands of users.
- Implemented critical enhancements, including advanced caching strategies.

### CTO, BetCraft (2013 - 2015)
- Led technology strategy post-Series A funding, guiding the company through technological advancement and network expansion.
- Improved platform performance and expanded market reach through strategic initiatives and partnerships.

## Education
- MBA in Information Technology, London Business School
- Advanced Leadership Techniques Certification, University of London
- Data Science Specialization Certification, Coursera (Johns Hopkins University)
- B.Sc. in Computer Science, University of Edinburgh

Noah Williams is an ideal candidate for senior executive roles, combining technical expertise with strategic leadership in technology companies.

- Dislplay the generated `interview_materials.md` file.

In [18]:
display(Markdown("./interview_materials.md"))

Based on the tailored resume of Noah Williams and the job requirements, here are the potential interview questions and talking points for the candidate to prepare for the initial interview:

1. Can you elaborate on your experience in transforming engineering divisions into revenue pillars, as mentioned in your work at DataKernel?
2. How have you integrated cutting-edge AI technologies and scalable databases to enhance product capabilities and market positioning?
3. In what ways have you fostered skill development and achieved strategic project goals within your teams?
4. Could you share insights into your role in defining long-term strategic initiatives and setting industry benchmarks in adopting AI technologies?
5. Describe your approach to leading diverse engineering teams across multiple time zones and the strategies you use to ensure transparency and mutual respect.
6. How do you recruit, mentor, and retain top talent to create a culture of continuous improvement and high performance within your teams?
7. Can you walk us through your experience in founding and leading InnovPet, particularly in developing IoT solutions for pet care and overseeing successful market entry?
8. What was your role in setting up an advisory board, establishing overseas production facilities, and navigating the initial funding phase for InnovPet?
9. Share examples of how you formulated and executed strategic plans to enhance inter-departmental coordination and trust at EliteDevs.
10. How did you manage multiple engineering teams, balancing productivity with innovation, and aligning goal-setting frameworks with company goals at EliteDevs?
11. Discuss your leadership in increasing company revenue by 500% within two years at PrintPack through the integration of data analytics and predictive modeling.
12. What was the impact of developing and optimizing a central API at DriveAI, and how did it enhance system performance and user satisfaction?
13. Explain your role in leading technology strategy post-Series A funding at BetCraft, including improving platform performance and expanding market reach through strategic initiatives and partnerships.
14. How did your educational background in Computer Science, an MBA in Information Technology, and certifications in Advanced Leadership Techniques and Data Science prepare you for leadership roles in the tech industry?
15. How do you plan to leverage your experience in Ruby, Python, JavaScript, TypeScript, and Elixir, along with expertise in data science and machine learning, to drive innovation and growth in our organization?

These questions and talking points aim to help the candidate showcase their relevant experience, skills, and achievements in alignment with the job requirements outlined for the role of a Full Stack Developer with expertise in multiple programming languages and frameworks, AI/ML models, big data tools, and consumer applications.